In [1]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, InputLayer, Dropout, Conv1D, MaxPool1D, Input, LayerNormalization, Reshape, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras import regularizers
from sklearn.utils import class_weight
import statistics as st
from sklearn.preprocessing import StandardScaler
from numpy import genfromtxt
from tensorflow.keras.layers import BatchNormalization
from sklearn.preprocessing import FunctionTransformer
from numpy import dstack
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import random
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tensorflow.keras import utils as np_utils

# Dataset preparation 

In [2]:
S_PATH = "dataset/ECE542_sp2022_Project_TerrainRecognition/"  #'/content/drive/MyDrive/ECE542/CompetitiveProject/ECE542_sp2022_Project_TerrainRecognition'
PATH = S_PATH +'/TrainingData/'
TEST_PATH = S_PATH + "./TestData/"
test_subject = 6

In [3]:
files = []
for filename in os.listdir(PATH):
    files.append(filename)
files = sorted(files, key = lambda x: (int(x.split('_')[1]),int(x.split('_')[2]), x.split('_')[4] ))
files_train = list(filter(lambda x: int(x.split('_')[1]) not in  [test_subject], files))
files_train = [os.path.join(PATH, i) for i in files_train]
files_test = list(filter(lambda x: int(x.split('_')[1]) == test_subject, files))
files_test = [os.path.join(PATH, i) for i in files_test]

In [4]:
files_train_x_path = files_train[1::4]
files_test_x_path = files_test[1::4]
files_train_y_path = files_train[3::4]
files_test_y_path = files_test[3::4]
files_train_x = [pd.read_csv(i) for i in files_train_x_path]
files_test_x = [pd.read_csv(i) for i in files_test_x_path]
files_train_y = [pd.read_csv(i) for i in files_train_y_path]
files_test_y = [pd.read_csv(i) for i in files_test_y_path]
# files_train_x, files_test_x, files_train_y, files_test_y

In [5]:
def generate_splits(all_files_paths, all_files):
    continue_loop = True
    threshold = 0.005
    all_files = pd.concat(all_files)
    _, files_split_dist = np.unique(all_files, return_counts=True)
    files_split_dist = np.divide(files_split_dist, np.max(files_split_dist))
    while continue_loop:
        random_seed = np.random.randint(0, (2 ** 31) - 1)
        np.random.seed(random_seed)
        random.seed(random_seed)
        files_train_y_split_1 = random.sample(all_files_paths, int(len(all_files_paths) * 0.8))
        files_train_y_split_2 = [i for i in all_files_paths if i not in files_train_y_split_1]
        files_split_1_val = pd.concat([pd.read_csv(i , header=None) for i in files_train_y_split_1])
        files_split_2_val = pd.concat([pd.read_csv(i , header=None) for i in files_train_y_split_2])
        _, files_split_1_dist = np.unique(files_split_1_val, return_counts=True)
        files_split_1_dist = np.divide(files_split_1_dist, np.max(files_split_1_dist))
        _, files_split_2_dist = np.unique(files_split_2_val, return_counts=True)
        files_split_2_dist = np.divide(files_split_2_dist, np.max(files_split_2_dist))
        if ((files_split_dist[0] - threshold <= files_split_1_dist[0] <= files_split_dist[0] + threshold) and
            (files_split_dist[1] - threshold <= files_split_1_dist[1] <= files_split_dist[1] + threshold) and
            (files_split_dist[2] - threshold <= files_split_1_dist[2] <= files_split_dist[2] + threshold) and
            (files_split_dist[3] - threshold <= files_split_1_dist[3] <= files_split_dist[3] + threshold) and
            (files_split_dist[0] - threshold <= files_split_2_dist[0] <= files_split_dist[0] + threshold) and
            (files_split_dist[1] - threshold <= files_split_2_dist[1] <= files_split_dist[1] + threshold) and
            (files_split_dist[2] - threshold <= files_split_2_dist[2] <= files_split_dist[2] + threshold) and
            (files_split_dist[3] - threshold <= files_split_2_dist[3] <= files_split_dist[3] + threshold)):
                print(files_split_1_dist, files_split_2_dist, files_split_dist)
                return sorted(files_train_y_split_1), sorted(files_train_y_split_2), random_seed


In [6]:
y_train_files, y_val_files, random_seed = generate_splits(files_train_y_path, files_train_y)

[1.         0.05549903 0.07357915 0.21037401] [1.         0.05648564 0.0787299  0.21565654] [1.         0.0557347  0.07478412 0.21162566]


In [7]:
y_train_file_names = ["_".join(i.split("/")[-1].split("__")[: -1]) for i in y_train_files]
y_val_file_names = ["_".join(i.split("/")[-1].split("__")[: -1]) for i in y_val_files]

In [8]:
x_train_files = sorted([i for i in files_train_x_path if "_".join(i.split("/")[-1].split("__")[: -1]) in y_train_file_names])
x_val_files = sorted([i for i in files_train_x_path if "_".join(i.split("/")[-1].split("__")[: -1]) in y_val_file_names])

In [9]:
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]


def mode(a, axis=0):
    scores = np.unique(np.ravel(a))       # get ALL unique values
    testshape = list(a.shape)
    testshape[axis] = 1
    oldmostfreq = np.zeros(testshape)
    oldcounts = np.zeros(testshape)

    for score in scores:
        template = (a == score)
        counts = np.expand_dims(np.sum(template, axis),axis)
        mostfrequent = np.where(counts > oldcounts, score, oldmostfreq)
        oldcounts = np.maximum(counts, oldcounts)
        oldmostfreq = mostfrequent

    return mostfrequent, oldcounts


def preprocessing(x_files, y_files, window_size=30):
    final_x = []
    final_y = []
    for i in range(len(y_files)):
        X_df = pd.read_csv(x_files[i])
        y_df = pd.read_csv(y_files[i])
        a = []
        for i in range(0, len(y_df)):
            a += [y_df['0'][i]] * 4
        upsampled_df = pd.DataFrame(a)
        diff = X_df.shape[0] - upsampled_df.shape[0]
        X_df = X_df.iloc[:-diff,:]

        sc=StandardScaler()
        X_df=sc.fit_transform(X_df)

        x_arr=[]
        y_arr=[]

        for j in range(len(X_df)-30):
            x_arr.append(X_df[j: (j + window_size)]) 
            y_arr.append(mode(upsampled_df.iloc[j: (j + window_size)])[0])
        final_x.append(np.array(x_arr))
        final_y.append(np.array(y_arr).reshape(-1,1))
    return final_x, final_y

In [10]:
trainX, trainY = preprocessing(x_train_files, y_train_files)
trainX, trainY = np.concatenate(trainX), np.concatenate(trainY)
valX, valY = preprocessing(x_train_files, y_train_files)
valX, valY = np.concatenate(valX), np.concatenate(valY)
testX, testY = preprocessing(files_test_x_path, files_test_y_path)
testX, testY = np.concatenate(testX), np.concatenate(testY)

In [ ]:
label_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(trainY), y=trainY.ravel())
label_weights = {i:label_weights[i] for i in range(len(label_weights))} # Create dictionary
print(label_weights)

In [ ]:
hot_y_train = np_utils.to_categorical(trainY)
hot_y_val = np_utils.to_categorical(valY)
# hot_y_test = np_utils.to_categorical(testY)

# Model training and inference

In [ ]:
def simpleLSTM(timestep, features, n_outputs, hidden_units=125):
    model_LSTM = Sequential()
    model_LSTM.add(InputLayer(input_shape=(timestep, features)))
    model_LSTM.add(LSTM(units=hidden_units, activation='relu'))
    model_LSTM.add(Dropout(0.5))
    model_LSTM.add(Dense(units = hidden_units, activation = 'relu'))
    model_LSTM.add(Dense(n_outputs, activation='softmax')) # output layer
    return model_LSTM


def simpleLSTM2(timestep, features, n_outputs, hidden_units=125):
    x_in = Input(shape=(timestep, features))
    x = LSTM(units=hidden_units, activation='relu', kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001), bias_regularizer=l2(0.001))(x_in)
    x = LayerNormalization()(x) 
    x = Dropout(0.5)(x)
    x = Dense(units = hidden_units, activation = 'relu', kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))(x)
    x = LayerNormalization()(x)
    x = Dense(n_outputs, activation='softmax')(x)
    model_LSTM = Model(inputs=x_in, outputs=x)
    return model_LSTM



def LSTMConv(timestep, features, n_outputs, hidden_units=125, filters=64, kernel_size=1):
    x_in = Input(shape=(timestep, features))
    x = LSTM(units=hidden_units, activation='relu')(x_in)
    x = LayerNormalization()(x) 
    x = Dropout(0.5)(x)
    x = Reshape((1, 1, hidden_units))(x)
    print(x.shape)
    x = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same')(x) 
    x = Flatten()(x) 
    x = Dense(units=hidden_units, activation='relu')(x)
    x = LayerNormalization()(x)
    x = Dense(n_outputs, activation='softmax')(x)
    model = Model(inputs=x_in, outputs=x)
    return model


In [ ]:
timestep, features, n_outputs = trainX.shape[1], trainX.shape[2], 4
# model = simpleLSTM(timestep, features, n_outputs, 125)
model = simpleLSTM2(timestep, features, n_outputs, 125)
# model = LSTMConv(timestep, features, n_outputs, hidden_units=125, filters=64, kernel_size=3)

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=3, min_lr=0.000001
    ),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1),
]
loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05)
optimizer= Adam(learning_rate=0.001)
metrics=['accuracy']
model.compile(optimizer, loss, metrics)

In [ ]:
history = model.fit(trainX, hot_y_train, epochs = 50, batch_size = 256,
                    validation_data = (valX, hot_y_val), class_weight = label_weights,
                    verbose = 1, shuffle = True, callbacks=callbacks)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'], color='red')
plt.plot(history.history['val_accuracy'], color='blue')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'], color='red')
plt.plot(history.history['val_loss'], color='blue')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
model_weights = r"C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\terrain_prediction\best_model.h5"
model.load_weights(model_weights)
y_pred = model.predict(testX, batch_size = 256, verbose = 1)
y_test_bool = np.argmax(y_pred, axis = 1)
print(classification_report(testY, y_test_bool))
print(confusion_matrix(testY, y_test_bool))

# Prediction generation on actual test set

In [ ]:
test_dataset_folder = r"C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\terrain_prediction\dataset\ECE542_sp2022_Project_TerrainRecognition\TestData"
save_folder = r"C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\Models\tf_experiments\6"

In [ ]:
def predict(model, test_dataset_folder, file_name_x, file_name_y, save_name):
    input_data = pd.read_csv(os.path.join(test_dataset_folder, file_name_x))
    sc = StandardScaler()
    df = sc.fit_transform(input_data)
    y_frame = pd.read_csv(os.path.join(test_dataset_folder, file_name_y))
    addl_rows = y_frame.shape[0] * 4 - df.shape[0] + 30
    addl_rows_df = pd.DataFrame(df[-addl_rows:])
    df = pd.DataFrame(df)
    df = df.append(addl_rows_df)
    X_values = []
    for i in range(0, len(df) - 30, 1):
        value = df.iloc[i:(i + 30)].values
        X_values.append(value)
    X_test = np.array(X_values)
    y_test = model.predict(X_test, batch_size = 256, verbose = 1)
    y_test_class = np.argmax(y_test, axis = 1)
    output_actual = []
    for i in range(0, y_test_class.shape[0], 4):
        a = list(y_test_class[i:i + 4])
        output_actual.append(max(a, key = a.count))
    y_actual = np.array(output_actual)
    print(y_actual.size)
    y_series = pd.Series(y_actual)
    y_series.to_csv(os.path.join(save_folder, save_name),index=False, header=None)

In [ ]:
model_weights = r"C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\terrain_prediction\best_model.h5"
model.load_weights(model_weights)

In [ ]:
predict(model, test_dataset_folder, "subject_009_01__x.csv", "subject_009_01__y_time.csv", "subject_009_01__y.csv")
predict(model, test_dataset_folder, "subject_009_01__x.csv", "subject_010_01__y_time.csv", "subject_010_01__y.csv")
predict(model, test_dataset_folder, "subject_009_01__x.csv", "subject_011_01__y_time.csv", "subject_011_01__y.csv")
predict(model, test_dataset_folder, "subject_009_01__x.csv", "subject_012_01__y_time.csv", "subject_012_01__y.csv")

# compare 2 models (discrepency)

In [ ]:
import os
import pandas as pd

In [ ]:
# target_files_folder = r"C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\Models\C3.2_submission"
target_files_folder = r"C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\Results_from_Sai"
predicted_files_folder = r"C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\Models\tf_experiments\6"
# predicted_files_folder = r"C:\Users\sumuk\OneDrive\Desktop\NCSU_related\Courses_and_stuff\Courses_and_stuff\NCSU_courses_and_books\ECE_542\CompetitiveProject\Models\C3.2_submission"

In [ ]:
target_files_path = [os.path.join(target_files_folder, i) for i in sorted(os.listdir(target_files_folder))]
predicted_files_path = [os.path.join(predicted_files_folder, i) for i in sorted(os.listdir(predicted_files_folder))]
target_files = [pd.read_csv(i, header=None) for i in target_files_path]
predicted_files = [pd.read_csv(i, header=None) for i in predicted_files_path]

In [ ]:
[len(i) for i in predicted_files], [len(i) for i in target_files]

In [ ]:
cms = []
for i, j in zip(target_files, predicted_files):
    target_values = list(i.values)
    predicted_values = list(j.values)
    print(len(target_values), len(predicted_values))
    cms.append(confusion_matrix(y_true=target_values, y_pred=predicted_values))

In [ ]:
cms[0]

In [ ]:
cms[1]

In [ ]:
cms[2]

In [ ]:
cms[3]